In [1]:
# Import libraries
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib
#matplotlib.use('qtagg')
import xarray as xr
import numpy as np
from netCDF4 import Dataset
import IPython.display
import cmocean.cm as cmo
import cartopy.crs as ccrs
import cartopy.feature as cft
#import cosima_cookbook as cc
#import cosima-cookbook as cc
#from mpl_toolkits.basemap import Basemap
#import basemap as bs
from os import listdir
from os.path import isfile, join
import math
import time
import sys
#from plotting_tools import *
import glob
from PIL import Image

from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.colors as colors
import matplotlib as mpl
import pandas as pd
import matplotlib.path as mpath

def ProgressBar(Total, Progress, BarLength=20, ProgressIcon="#", BarIcon="-"):
    try:
        # You can't have a progress bar with zero or negative length.
        if BarLength <1:
            BarLength = 20
        # Use status variable for going to the next line after progress completion.
        Status = ""
        # Calcuting progress between 0 and 1 for percentage.
        Progress = float(Progress) / float(Total)
        # Doing this conditions at final progressing.
        if Progress >= 1.:
            Progress = 1
            Status = "\r\n"    # Going to the next line
        # Calculating how many places should be filled
        Block = int(round(BarLength * Progress))
        # Show this
        Bar = "[{}] {:.0f}% {}".format(ProgressIcon * Block + BarIcon * (BarLength - Block), round(Progress * 100, 0), Status)
        return Bar
    except:
        return "ERROR"


def ShowBar(Bar):
    sys.stdout.write(Bar)
    sys.stdout.flush()
    
    
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs 

def add_cyclic_point_to_dataset(ds):
    # Load data
    data = ds["t2m"]

    # Generate data with cyclic point and generate longitude with cyclic point
    cyclic_data, cyclic_longitude = add_cyclic_point(data.values, coord=data['longitude'])

    # Create new coords that will be used in creation of new dataset
    # Replicate coords of existing dataset and replace longitude with cyclic longitude
    coords = {dim: data.coords[dim] for dim in data.dims}
    coords["longitude"] = cyclic_longitude

    new_ds = xr.Dataset(
        data_vars={
            "t2m": xr.DataArray(cyclic_data, dims=data.dims, coords=coords)
        })
    return new_ds 



### Functions


In [2]:
def make_gif(frame_folder, gifname, yr , dframe):
    frames = [Image.open(image) for image in sorted(glob.glob(f"{frame_folder}/*.png"))[0:len(glob.glob(f"{frame_folder}/*.png")):dframe]]
    frame_one = frames[0]
    frame_one.save(gifname + yr + ".gif", format="GIF", append_images=frames,
               save_all=True, duration=100, loop=0)

    
def make_map(ds, ax):
#     new_ds = add_cyclic_point_to_dataset(ds)
#     new_ds["k"].shape
#     # (1801, 3601)

#     difference = ds["k"].sel(longitude=0, method="nearest") - new_ds["k"].sel(longitude=360, method="nearest")
#     difference = difference.fillna(0)  # fill nan values with 0
#     any(difference != 0)
#     # False
#     projection = ccrs.SouthPolarStereo() 
    
    cs = ax.pcolormesh(ds['LON'][:,:], ds['LAT'][:,:],  ds['k'][i,:,:],
            transform=ccrs.PlateCarree(),
            cmap=cmap,
            norm=norm,
            shading='auto')     

    ax.set_extent(coord_lims, ccrs.PlateCarree())

    # Add coastlines
#    ax.coastlines()
    land_50m = cft.NaturalEarthFeature('physical', 'land', '50m',
                                 edgecolor='None', facecolor='gray', linewidth=0.5)
    ax.add_feature(land_50m)
    
    
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax.set_boundary(circle, transform=ax.transAxes)


    # Adjust the location of the subplots on the page to make room for the colorbar
#    fig.subplots_adjust(bottom=0.0, top=0.9, left=0.05, right=0.9,
#                        wspace=0.1, hspace=0.5)

    # Add a colorbar axis at the bottom of the graph
#    cbar_ax = fig.add_axes([0.925, 0.165, 0.01, 0.57])

    # Draw the colorbar
#    cbar=fig.colorbar(cs, cax=cbar_ax,orientation='vertical', label='Cluster number') 
    
    return ax


def add_cyclic_point_to_dataset(ds):
    # Load data
    data = ds["k"]

    # Generate data with cyclic point and generate longitude with cyclic point
    cyclic_data, cyclic_longitude = add_cyclic_point(data.values, coord=data['LON'])

    # Create new coords that will be used in creation of new dataset
    # Replicate coords of existing dataset and replace longitude with cyclic longitude
    coords = {dim: data.coords[dim] for dim in data.dims}
    coords["LON"] = cyclic_longitude

    new_ds = xr.Dataset(
        data_vars={
            "k": xr.DataArray(cyclic_data, dims=data.dims, coords=coords)
        })
    return new_ds 

###################################################################################################

    

# Main script




In [ ]:
# main subroutine
#if __name__ == "__main__":
land_50m = cft.NaturalEarthFeature('physical', 'land', '50m',
                                edgecolor='black', facecolor='gray', linewidth=0.5)

year = '2004'
var_name = 'aice'
variable_list = get_var_list(var_name)
case = 'wim'
hemisphere = "south"
#date = '2006-09-30'
machine = 'maths1'
#filename =  get_machine_dir(machine)+case+'/history/iceh.'+date+'.nc'

mypath = get_machine_dir(machine)+case+'/history' #'/g/data/dy43/cice-dirs/runs/nowaves/history/' + year
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
filenames = sorted(onlyfiles)
#filenames = ''
plot_dir = year + '/' + var_name #'/scratch/dy43/nd0349/plotting/' + year + '/' + var_name
filenames = filenames[0:730:1]

print(filenames)
for i, f in enumerate(filenames):
    #progressBar = "\rProgress: " + ProgressBar(len(filenames), i, 20, '#', '.')
    #ShowBar(progressBar)
    filename = mypath + '/' + f
    ds = xr.open_dataset(filename)

    fig = plt.figure()#figsize=(15,8))
    
    LN = ds.TLON.values
    LT = ds.TLAT.values

    projection = ccrs.SouthPolarStereo()
        # ccrs.SouthPolarStereo()
        # ccrs.Robinson()

   # print('Plotting:')
    for counter, exp in enumerate(variable_list):
       # ax = plt.subplot(2, math.ceil(len(variable_list)/2), counter+1, projection=projection, label='{0}'.format(counter))
        ax = plt.subplot(1,1, counter+1, projection=projection, label='{0}'.format(counter))
        #print('     ' + exp)
        ax.coastlines(resolution='50m')
        ax.add_feature(land_50m)
        ax.set_extent([-180, 180, -40, -90], ccrs.PlateCarree())
        #gl = ax.gridlines(draw_labels=False)
        ax= plot_map(ax,exp,ds,LN,LT,hemisphere)
        #if exp == 'vocn' or exp == 'uocn' or exp == 'vatm' or exp == 'uatm' or exp == 'uvel' or exp == 'vvel':
        #    p1 = ax.pcolormesh(LN,LT,ds[exp][0,:,:], 
        #                    transform=ccrs.PlateCarree(),
        #                    cmap='RdBu_r', norm=colors.CenteredNorm())
        #if exp == 'aice':
        #    p1 = ax.pcolormesh(LN,LT,ds[exp][0,:,:], 
        #                    transform=ccrs.PlateCarree(),
        #                    cmap=cmo.ice, vmin=0, vmax = 1)
        #else:
        #     p1 = ax.pcolormesh(LN,LT,ds[exp][0,:,:], 
        #                     transform=ccrs.PlateCarree())



        
        #p1 = ax.pcolormesh(ds['TLON'][:,:],ds['TLAT'][:,:],ds[exp][0,:,:],#.TLAT.ni, ds[exp].TLON.nj,
                        #transform=ccrs.PlateCarree(),
        #                   cmap='RdBu_r', vmin=-0.8, vmax=0.8)
        ax.set_title(variable_list[counter])

        #ax_cb = plt.axes([0.85, 0.15, 0.03, 0.7])
        #cb = plt.colorbar(p1, cax=ax_cb, orientation='vertical')
        #ax_cb = plt.axes([0.85, 0.15, 0.03, 0.7])
        #cb = plt.colorbar(p1, orientation='vertical')
        #cb.ax.set_ylabel('Zonal velocity (m/s)');

    progressBar = "\rProgress: " + ProgressBar(len(filenames), i+1, 20, '#', '.')
    ShowBar(progressBar)


    #print('Saving figure...')
    temp = filename.partition('.')[2]
    fig.suptitle(temp.partition('.')[0], fontsize=15)
    plt.savefig(plot_dir + '/' + temp.partition('.')[0]+'-'+var_name+'.png', dpi = 300)  
    # plt.savefig(temp.partition('.')[0]+'-'+var_name+'.png', dpi = 100)  
   # print('Image saved!')

make_gif(plot_dir, var_name, year)



In [ ]:


year = '2004'
var_name = 'aice'
variable_list = get_var_list(var_name)
case = 'wim'
hemisphere = "south"
#date = '2006-09-30'
machine = 'maths1'
#filename =  get_machine_dir(machine)+case+'/history/iceh.'+date+'.nc'

mypath = get_machine_dir(machine)+case+'/history' #'/g/data/dy43/cice-dirs/runs/nowaves/history/' + year
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
filenames = sorted(onlyfiles)
#filenames = ''
plot_dir = year + '/' + var_name #'/scratch/dy43/nd0349/plotting/' + year + '/' + var_name

make_gif(plot_dir, var_name, year)

In [ ]:


# main subroutine
if __name__ == "__main__":
    land_50m = cft.NaturalEarthFeature('physical', 'land', '50m',
                                    edgecolor='black', facecolor='gray', linewidth=0.5)

    year = '2004'
    var_name = 'ocn'
    variable_list = get_var_list(var_name)
    case = 'wim'
    date = '2006-09-30'
    machine = 'maths1'
    #filename =  get_machine_dir(machine)+case+'/history/iceh.'+date+'.nc'

    mypath = get_machine_dir(machine)+case+'/history' #'/g/data/dy43/cice-dirs/runs/nowaves/history/' + year
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    filenames = sorted(onlyfiles)
    #filenames = ''
    plot_dir = year + '/' + var_name #'/scratch/dy43/nd0349/plotting/' + year + '/' + var_name
    filenames = filenames[0:10]
    for i, f in enumerate(filenames):
        filename = mypath + '/' + f
        ds = xr.open_dataset(filename)

        fig = plt.figure()#figsize=(15,8))
        
        LN = ds.TLON.values
        LT = ds.TLAT.values

        projection = ccrs.SouthPolarStereo()
            # ccrs.SouthPolarStereo()
            # ccrs.Robinson()

        print('Plotting:')
        for counter, exp in enumerate(variable_list):
            ax = plt.subplot(2, math.ceil(len(variable_list)/2), counter+1, projection=projection, label='{0}'.format(counter))
            print('     ' + exp)
            ax.coastlines(resolution='50m')
            ax.add_feature(land_50m)
            ax.set_extent([-180, 180, -40, -90], ccrs.PlateCarree())
            #gl = ax.gridlines(draw_labels=False)
            
            if exp == 'vocn' or exp == 'uocn' or exp == 'vatm' or exp == 'uatm' or exp == 'uvel' or exp == 'vvel':
                p1 = ax.pcolormesh(LN,LT,ds[exp][0,:,:], 
                                transform=ccrs.PlateCarree(),
                                cmap='RdBu_r', norm=colors.CenteredNorm())
            if exp == 'aice':
                p1 = ax.pcolormesh(LN,LT,ds[exp][0,:,:], 
                                transform=ccrs.PlateCarree(),
                                cmap=cmo.ice, norm=colors.CenteredNorm())
            else:
                p1 = ax.pcolormesh(LN,LT,ds[exp][0,:,:], 
                                transform=ccrs.PlateCarree())



            
            #p1 = ax.pcolormesh(ds['TLON'][:,:],ds['TLAT'][:,:],ds[exp][0,:,:],#.TLAT.ni, ds[exp].TLON.nj,
                            #transform=ccrs.PlateCarree(),
            #                   cmap='RdBu_r', vmin=-0.8, vmax=0.8)
            ax.set_title(variable_list[counter])

            #ax_cb = plt.axes([0.85, 0.15, 0.03, 0.7])
            #cb = plt.colorbar(p1, cax=ax_cb, orientation='vertical')
            #ax_cb = plt.axes([0.85, 0.15, 0.03, 0.7])
            cb = plt.colorbar(p1, orientation='vertical')
            #cb.ax.set_ylabel('Zonal velocity (m/s)');




        print('Saving figure...')
        temp = filename.partition('.')[2]
        fig.suptitle(temp.partition('.')[0], fontsize=15)
        plt.savefig(plot_dir + '/' + temp.partition('.')[0]+'-'+var_name+'.png', dpi = 100)  
        print('Image saved!')

    make_gif(plot_dir, var_name, year)


In [4]:
#print(matplotlib.font_manager.get_font_names())

In [ ]:
# Make a k-means animation
var = "kmean"
year = 2019

plot_dir = '/home/566/nd0349/notebooks/figures/gif'
filename = '/g/data/ia40/sea-ice-classification/kmean_2019.nc'
ds = xr.open_dataset(filename)

n_days, n_lat, n_lon = ds['k'].shape

dates = pd.date_range(start="2019-01-01",end="2019-12-31", periods=365)
title_dates = dates.strftime("%d/%m/%Y")

fig = plt.figure(figsize=(19.2, 10.8))
ax = plt.subplot(projection=ccrs.SouthPolarStereo())

plt.rcParams["font.family"] = "serif"

# Define the contour levels to use in plt.contourf
clevs=np.arange(0,5,1)
coord_lims = [-180, 180, -50, -90]

# Make a colormap
cmap = mpl.colors.ListedColormap([(0.9805, 0.5000, 0.4453, 1.0), (0.4416, 0.7490, 0.4322, 1.0), (0.3639, 0.5755, 0.748, 1.0)])
# define the bins and normalize
bounds = np.linspace(0, 3, 4)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

date_x = 0
date_y = -60

for i, f in enumerate(dates):
    progressBar = "\rProgress: " + ProgressBar(len(dates), i, 20, '#', '.')
    ShowBar(progressBar)

    ax = make_map(ds, ax=ax)

#    ax.set_title(f,
#             fontsize = 14)

    ax.text(date_x, date_y, 
            f"{title_dates[i]}", 
            color='black',
            fontsize = 15,
            horizontalalignment = 'center',
            transform=ccrs.PlateCarree())
    plt.savefig(plot_dir + '/' + var+'-'+ f.strftime("%Y-%m-%d") +'.png', dpi = 300)
    
    # Clear the axis to save memory
    ax.clear() 
    #plt.show()
    
sys.stdout.flush()
print("Making gif...")
gifname = plot_dir + '/' + var
make_gif(plot_dir, gifname, str(year), 1)
print("Done!")


In [ ]:
var = "kmean"
year = 2019
plot_dir = '/home/566/nd0349/notebooks/figures/gif'
gifname = plot_dir + '/' + var
make_gif(plot_dir, gifname, str(year), 2)

In [60]:
f.strftime("%Y/%m/%d")

'2019/01/01'

In [3]:
# USES WAYY TOO MUCH MEMORY

import cv2
import os

image_folder = '/home/566/nd0349/notebooks/figures/gif'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()